# Sheath Expansion Simulations Results - Analysis Round 1
First from a series of notebooks analysing the sheath expansions simulations in a manner similar to Bergmann in his 2002 paper. This one combines IVs together into a single xarray object for easy later manipulation. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm

In [2]:
importlib.reload(fts)

<module 'flopter.core.fitters' from '/home/jleland/coding/projects/flopter/flopter/core/fitters.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [3]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [4]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [7]:
skippable_scans = {
    'marconi/spice2/sheath_exp/angled',
#     'marconi/spice2/sheath_exp/angled_1',
#     'marconi/spice2/sheath_exp/angled_longtail_10',
#     'marconi/spice2/sheath_exp/angled_recessed',
#     'marconi/spice2/sheath_exp/angled_recessed_1',
#     'marconi/spice2/sheath_exp/angled_semi-recessed',
#     'marconi/spice2/sheath_exp/angled_semi-recessed_1',
#     'marconi/spice2/sheath_exp/flat',
#     'marconi/spice2/sheath_exp/flat_1',
#     'marconi/spice2/sheath_exp/flat_recessed',
#     'marconi/spice2/sheath_exp/flat_recessed_1',
#     'marconi/spice2/sheath_exp/flat_semi-recessed',
#     'marconi/spice2/sheath_exp/flat_semi-recessed_1',
#     'marconi/spice2/sheath_exp/semi-angled',
#     'marconi/spice2/sheath_exp/semi-angled_1',
#     'marconi/spice2/sheath_exp/semi-angled_recessed',
#     'marconi/spice2/sheath_exp/semi-angled_recessed_1',
#     'marconi/spice2/sheath_exp/semi-angled_semi-recessed',
#     'marconi/spice2/sheath_exp/semi-angled_semi-recessed_1',
    

} 
single_sims = set()

In [8]:
scans_searchstr = '*/*/new_sheath_exp/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)


skippable = {}
scans = list(scans)
scans.sort()

In [9]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/new_sheath_exp/angled_flush
	[0,0]: angled_flush/alpha_yz_-10.0
	[0,1]: angled_flush/alpha_yz_-12.0
	[0,2]: angled_flush/alpha_yz_-2.0
	[0,3]: angled_flush/alpha_yz_-20.0
	[0,4]: angled_flush/alpha_yz_-3.0
	[0,5]: angled_flush/alpha_yz_-30.0
	[0,6]: angled_flush/alpha_yz_-4.0
	[0,7]: angled_flush/alpha_yz_-5.0
	[0,8]: angled_flush/alpha_yz_-6.0
	[0,9]: angled_flush/alpha_yz_-8.0
[1]: marconi/spice2/new_sheath_exp/angled_recessed
	[1,0]: angled_recessed/alpha_yz_-10.0
	[1,1]: angled_recessed/alpha_yz_-12.0
	[1,2]: angled_recessed/alpha_yz_-2.0
	[1,3]: angled_recessed/alpha_yz_-20.0
	[1,4]: angled_recessed/alpha_yz_-3.0
	[1,5]: angled_recessed/alpha_yz_-30.0
	[1,6]: angled_recessed/alpha_yz_-4.0
	[1,7]: angled_recessed/alpha_yz_-5.0
	[1,8]: angled_recessed/alpha_yz_-6.0
	[1,9]: angled_recessed/alpha_yz_-8.0
[2]: marconi/spice2/new_sheath_exp/flat_flush
	[2,0]: flat_flush/alpha_yz_-10.0
	[2,1]: flat_flush/alpha_yz_-12.0
	[2,2]: flat_flush/alpha_yz_-2.0
	[2,3]: flat_flu

---
## Function For Per-run Analysis

In [10]:
spl_path = spice_dir / all_run_dirs[scans[0]][1]
print(spl_path)


/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-12.0


In [11]:
splopter =  spl.Splopter(spl_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True)
print(splopter.tdata.dt)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-12.0) doesn't seem to be valid.
Continuing anyway.


IndexError: list index out of range

In [12]:
def homogenise_run(path):
    splopter =  spl.Splopter(path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                             store_dataframe_fl=True)
    splopter.parse_input()
    iv_data, raw_iv_data = splopter.homogenise(reduced_bin_fract=0.1)
    return splopter.iv_df, iv_data

def get_dummy_fit(iv_data, fitter):
    dummy_params = np.nan * np.zeros_like(fitter.default_values)
    dum_fit_data = fd.IVFitData(iv_data['V'], iv_data['I'], np.nan * np.zeros_like(iv_data['I']), 
                                dummy_params, dummy_params, fitter, 
                                sigma=iv_data['sigma'], chi2=np.nan, reduced_chi2=np.nan)
    return dum_fit_data

In [13]:
def straight_iv_fit(iv_data, cutoff=-2):
    fitter = fts.FullIVFitter()
    iv_region = np.where(iv_data['V'] <= -2)
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    try:        
        fit_data = fitter.fit_iv_data(fit_iv_data, sigma=fit_iv_data['sigma'])
    except ValueError:
        print('CAUGHT: straight fit error')
        fit_data = get_dummy_fit(fit_iv_data, fitter)
    
    return fit_data
        
def multi_iv_fit(iv_data, sat_region=-6, **kwargs):
    try:
        fit_data = iv_data.multi_fit(sat_region=sat_region, **kwargs)
    except ValueError:
        print('CAUGHT: multi fit error')
        fit_data = get_dummy_fit(iv_data, fts.FullIVFitter())
        
    return fit_data


def norm_ion_fit(iv_data_orig, sigma=None):
    iv_data = iv_data_orig.copy()
    if sigma is None:
        iv_data['sigma'] = iv_data['sigma'] * 0.5
    else:
        iv_data['sigma'] = sigma
    
    V_f = fts.IVFitter.find_floating_pot(iv_data['V'][:-1], iv_data['I'][:-1])
#     V_f = iv_data.get_vf()
    
#     sim_ds = sim_ds.sel(voltage=slice(-9.95, V_f))
    iv_region = np.where(iv_data['V'] <= V_f)
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    i_fitter = fts.IonCurrentSEFitter()
           
    try:
        fit_data = i_fitter.fit(np.float_power(np.abs(fit_iv_data['V'] - V_f), .75), 
                                fit_iv_data['I_i'],
                                sigma=fit_iv_data['sigma'])
    except ValueError:
        print('CAUGHT: ion fit value error')
        fit_data = get_dummy_fit(fit_iv_data, i_fitter)
    except RuntimeError:
        print('CAUGHT: ion fit runtime error')
        fit_data = get_dummy_fit(fit_iv_data, i_fitter)
        
    return fit_data

def norm_full_fit(iv_data):
#     V_f = iv_data.get_vf()
    V_f = fts.IVFitter.find_floating_pot(iv_data['V'][:-1], iv_data['I'][:-1])
    
    iv_region = np.where(iv_data['V'] <= V_f)
    fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)
    
    voltage = fit_iv_data['V'] - V_f
    current = fit_iv_data['I']
    d_current = fit_iv_data['sigma']
    
    iv_fitter = fts.NormalisedIVFitter()
    try:
        fit_data = iv_fitter.fit(voltage, current, sigma=d_current)
    except ValueError:
        print('CAUGHT: norm full fit error')
        fit_data = get_dummy_fit(fit_iv_data, iv_fitter)
        
    return fit_data
    

In [14]:
# Test
iv_df, iv_data = homogenise_run(spl_path)

# iv.IVData.fractional_trim(iv_data, trim_beg=0.5).plot(ax=ax) #.multi_fit().plot(ax=ax)

fig, ax = plt.subplots()
# print(straight_iv_fit(iv_data).to_dict())
# multi_iv_fit(iv_data, plot_fl=True, mode=5, trimming_vals=(0.2, 0.0, 0.001), minimise_temp_fl=False).plot(ax=ax)
norm_full_fit(iv_data).plot(ax=ax)
norm_ion_fit(iv_data, sigma=iv_df['d_current_i']).plot(ax=ax)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-12.0) doesn't seem to be valid.
Continuing anyway.


IndexError: list index out of range

## Run analysis on subset of runs
To see how to combine them into a dataset

In [15]:
# import importlib
# importlib.reload(iv)
# importlib.reload(spl)

In [16]:
for scan in scans:
    for run_dir in all_run_dirs[scan]:
        run_path = spice_dir / run_dir
        bups = list(run_path.glob('backup*'))
        bups.sort()
        final_state = bups[-1].name
        
        print(run_dir)

marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-10.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-12.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-2.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-20.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-3.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-30.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-4.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-5.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-6.0
marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-8.0
marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-10.0
marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-12.0
marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-2.0
marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-20.0
marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-3.0
marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-30.0
marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-4.0
ma

In [17]:
iv_dfs = {}
fit_datas = {}
iv_datas = {}
failed_runs = []

In [18]:
scans_oi = scans[:2]

for scan in scans:
    for run_dir in all_run_dirs[scan]:
        run_path = spice_dir / run_dir
        
        bups = list(run_path.glob('backup*'))
        bups.sort()
        final_state_path = bups[-1]
        
        print(f'\n\n --- {run_dir} --- \n')
        if run_dir in iv_dfs:
            print("skipping...")
            continue
        
        if '-2.0' in run_dir:
            print(f'!! Skipping {run_dir}...')
            continue
        
        try:
#         iv_df, fit_data = analyse_run(run_path, mode=5, trimming_vals=(0.2, 0.0, 0.001))
            iv_df, iv_data = homogenise_run(final_state_path) #, minimise_temp_fl=False)

            iv_dfs[run_dir] = iv_df
            iv_datas[run_dir] = iv_data
        except Exception as e:
            failed_runs.append(run_dir)
            print(f'Failed on {run_dir}, {e}')



 --- marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-10.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-10.0/backup_20201029-0119) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)




 --- marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-12.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-12.0/backup_20201028-2311) doesn't seem to be valid.
Continuing anyway.
!! Skipping marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-2.0...


 --- marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-20.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-20.0/backup_20201028-1639) doesn't seem to be valid.
Continuing anyway.


 --- marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-3.0 --- 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-3.0/backup_20201104-0559) doesn't seem to be valid

In [19]:
failed_runs

[]

In [20]:
# testing an individual problem path

# p = pth.Path("/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/flat_recessed/alpha_yz_-1.0/backup_20200810-0213")
# p = pth.Path("/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/flat_recessed/alpha_yz_-1.0/backup_20200809-1816")
# splopter =  spl.Splopter(p, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
#                          store_dataframe_fl=True)
# splopter.parse_input()
# iv_data, raw_iv_data = splopter.homogenise()
# splopter.iv_df

In [21]:
# fig, ax = plt.subplots()

# for i in range(len(splopter.tdata.objectscurrente)):
#     ax.plot(splopter.tdata.objectscurrente[i][::100], label=i)
#     ax.legend()

In [22]:
def fd_to_ds(fit_data, fit_prefix='fit', extras=None):
    fit_ds = xr.Dataset(fit_data.to_dict()).drop(['raw_y', 'sigma', 'fit_y'])
    if extras is not None:
        for label, value in extras.items():
            fit_ds[label] = value
    fit_ds = fit_ds.assign(voltage_min=fit_ds.raw_x.min(), voltage_max=fit_ds.raw_x.max()).drop('raw_x').expand_dims(dim=['theta'])
    fit_ds = fit_ds.rename_vars({data_var: f'{fit_prefix}_{data_var}' for data_var in fit_ds.data_vars})
    
    return fit_ds

In [25]:
datasets = {}
probes = []
thetas = []


for scan in scans:
    for run_dir in all_run_dirs[scan]:
        if run_dir in failed_runs or run_dir not in iv_datas:
            print(f'Skipping {run_dir} as it failed...')
            continue
        fit_dss = []
        angle = float(run_dir.split('/')[-1].split('-')[-1])
        probe = run_dir.split('/')[-2].split('_1')[0]
        print(probe, angle)
        
        if angle not in thetas:
            thetas.append(angle)
        if probe not in probes:
            probes.append(probe)
        if probe not in datasets:
            datasets[probe] = []

        iv_data = iv_datas[run_dir]
        iv_df = iv_dfs[run_dir]
        
        iv_ds = iv_df.to_xarray().swap_dims({'index':'voltage'}).drop('index').expand_dims(dim=['theta'])
        
        
        str_iv_fit_ds = fd_to_ds(straight_iv_fit(iv_data), fit_prefix='str_iv', extras={'run_dir': run_dir})
#         multi_fit_ds = fd_to_ds(multi_iv_fit(iv_data), fit_prefix='multi')
        norm_iv_fit_ds = fd_to_ds(norm_full_fit(iv_data), fit_prefix='norm_iv')
        ion_fit_ds = fd_to_ds(norm_ion_fit(iv_data, sigma=iv_df['d_current_i']), fit_prefix='ion')
#         fit_ds = fd_to_ds(fit_datas[run_dir], fit_prefix='fit', )
        
        ds = xr.merge([str_iv_fit_ds, norm_iv_fit_ds, ion_fit_ds, iv_ds]).assign_coords(
            theta=xr.DataArray([angle], dims='theta')
        )
        datasets[probe].append(ds)        


angled_flush 10.0
angled_flush 12.0
Skipping marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-2.0 as it failed...
angled_flush 20.0
angled_flush 3.0
angled_flush 30.0
angled_flush 4.0
angled_flush 5.0
angled_flush 6.0
angled_flush 8.0
angled_recessed 10.0
angled_recessed 12.0
Skipping marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-2.0 as it failed...
angled_recessed 20.0
angled_recessed 3.0
angled_recessed 30.0
angled_recessed 4.0
angled_recessed 5.0
angled_recessed 6.0
angled_recessed 8.0
flat_flush 10.0
flat_flush 12.0
Skipping marconi/spice2/new_sheath_exp/flat_flush/alpha_yz_-2.0 as it failed...
flat_flush 20.0
flat_flush 3.0
flat_flush 30.0
flat_flush 4.0
flat_flush 5.0
flat_flush 6.0
flat_flush 8.0
flat_recessed 10.0
flat_recessed 12.0
Skipping marconi/spice2/new_sheath_exp/flat_recessed/alpha_yz_-2.0 as it failed...
flat_recessed 20.0
flat_recessed 3.0
CAUGHT: ion fit runtime error
flat_recessed 30.0
flat_recessed 4.0
flat_recessed 5.0
flat_recessed 6.0
flat_recesse

flat_recessed 1.0
- CAUGHT: straight fit error
- CAUGHT: norm full fit error
- CAUGHT: ion fit value error

flat_recessed 2.0
- CAUGHT: norm full fit error
- CAUGHT: ion fit value error

flat_recessed 3.0
- CAUGHT: ion fit runtime error

flat_semi-recessed 1.0
- CAUGHT: norm full fit error
- CAUGHT: ion fit value error

In [24]:
# iv_ds = iv_df.to_xarray().swap_dims({'index':'voltage'}).drop('index').expand_dims(dim=['angle', 'probe'])

# fit_ds = xr.Dataset(fit_data.to_dict()).drop(['raw_y', 'sigma', 'fit_y'])
# fit_ds = fit_ds.assign(voltage_min=fit_ds.raw_x.min(), voltage_max=fit_ds.raw_x.max()).drop('raw_x').expand_dims(dim=['angle', 'probe'])
# fit_ds = fit_ds.rename_vars({data_var: f'fit_{data_var}' for data_var in fit_ds.data_vars})

# ds = xr.merge([fit_ds, iv_ds]).assign_coords(
#     angle=xr.DataArray([angle], dims='angle'), 
#     probe=xr.DataArray([probe], dims='probe')
# )
# ds

In [26]:
probes

['angled_flush', 'angled_recessed', 'flat_flush', 'flat_recessed']

In [29]:
# probes_list = list(probes)
for i, probe in enumerate(probes):
    print(i, probe)

print()

for i, theta in enumerate(thetas):
    print(i, theta)

    

0 angled_flush
1 angled_recessed
2 flat_flush
3 flat_recessed

0 10.0
1 12.0
2 20.0
3 3.0
4 30.0
5 4.0
6 5.0
7 6.0
8 8.0


In [30]:
for probe, probe_list in datasets.items():
    print(probe)

angled_flush
angled_recessed
flat_flush
flat_recessed


In [70]:
datasets['angled_flush']

[<xarray.Dataset>
 Dimensions:               (theta: 1, voltage: 400)
 Coordinates:
   * voltage               (voltage) float64 -9.95 -9.9 -9.85 ... 9.9 9.95 10.0
   * theta                 (theta) float64 10.0
 Data variables:
     str_iv_chi2           (theta) float64 287.8
     str_iv_reduced_chi2   (theta) float64 1.857
     str_iv_I_sat          (theta) float64 -33.26
     str_iv_d_I_sat        (theta) float64 0.6944
     str_iv_a              (theta) float64 0.08216
     str_iv_d_a            (theta) float64 0.00695
     str_iv_T_e            (theta) float64 0.9199
     str_iv_d_T_e          (theta) float64 0.01497
     str_iv_V_f            (theta) float64 -3.381
     str_iv_d_V_f          (theta) float64 0.006945
     str_iv_run_dir        (theta) <U57 'marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-10.0'
     str_iv_voltage_min    (theta) float64 -9.95
     str_iv_voltage_max    (theta) float64 -2.05
     norm_iv_chi2          (theta) float64 285.7
     norm_iv_reduced_

In [31]:
# d_perps = xr.DataArray(
#     [0.0, 5.0e-4, 1.0e-3], 
#     dims=['recession'],
# ).assign_coords(recession_desc=('recession', ['flush', 'semi-recessed', 'recessed']))

# theta_p = xr.DataArray(
#     [0.0, 5, 10.0],
#     dims='theta_p'
# ).assign_coords(theta_p_desc=('theta_p', ['flat', 'semi-angled', 'angled']))


In [56]:
d_perps = xr.DataArray(
    [0.0, 1.0e-3], 
    dims='recession',
    coords={'recession':[0.0, 1.0e-3]}
).assign_coords(recession_desc=('recession', ['flush', 'recessed']))

theta_p = xr.DataArray(
    [0.0, 10.0],
    dims='theta_p',
    coords={'theta_p':[0.0, 10.0]}
).assign_coords(theta_p_desc=('theta_p', ['flat', 'angled']))


In [59]:
theta_datasets = []
for probe, probe_list in datasets.items():
#     theta_da = xr.DataArray(thetas, dims=['theta'])
    theta_combined_ds = xr.concat(probe_list, dim='theta').sortby('theta')
#     .assign_coords(
#         probe=xr.DataArray([d_perp], dims='recession')
#     )
    theta_datasets.append(theta_combined_ds)
    
# theta_p_datasets = [
#     xr.concat([theta_datasets[3], theta_datasets[6], theta_datasets[0]], dim=theta_p),
#     xr.concat([theta_datasets[5], theta_datasets[8], theta_datasets[2]], dim=theta_p),
#     xr.concat([theta_datasets[4], theta_datasets[7], theta_datasets[1]], dim=theta_p),
# ]
theta_p_datasets = [
    xr.concat([theta_datasets[2], theta_datasets[0]], dim=theta_p),
    xr.concat([theta_datasets[3], theta_datasets[1]], dim=theta_p),
]

    
# probe_da = xr.DataArray(list(probes), dims=['probe'])
# combined_ds = xr.concat(theta_datasets, dim='probe')
# combined_ds = xr.combine_nested(datasets_nl, concat_dim=['probe', 'theta'])
combined_ds = xr.concat(theta_p_datasets, dim=d_perps)
combined_ds

<xarray.Dataset>
Dimensions:               (recession: 2, theta: 9, theta_p: 2, voltage: 401)
Coordinates:
  * recession             (recession) float64 0.0 0.001
    theta_p_desc          (theta_p) <U6 'flat' 'angled'
  * theta_p               (theta_p) float64 0.0 10.0
  * theta                 (theta) float64 3.0 4.0 5.0 6.0 ... 12.0 20.0 30.0
  * voltage               (voltage) float64 -9.95 -9.9 -9.85 ... 9.95 10.0 10.05
    recession_desc        (recession) <U8 'flush' 'recessed'
Data variables:
    str_iv_chi2           (recession, theta_p, theta) float64 6.743e+03 ... 159.0
    str_iv_reduced_chi2   (recession, theta_p, theta) float64 43.5 ... 1.026
    str_iv_I_sat          (recession, theta_p, theta) float64 -11.35 ... -71.91
    str_iv_d_I_sat        (recession, theta_p, theta) float64 3.13 ... 1.296
    str_iv_a              (recession, theta_p, theta) float64 1.868e-15 ... 0.04223
    str_iv_d_a            (recession, theta_p, theta) float64 0.12 ... 0.005092
    str_iv_T_e            (recession, theta_p, theta) float64 1.995 ... 0.9402
    str_iv_d_T_e          (recession, theta_p, theta) float64 0.3035 ... 0.02061
    str_iv_V_f            (recession, theta_p, theta) float64 -4.721 ... -2.776
    str_iv_d_V_f          (recession, theta_p, theta) float64 0.05557 ... 0.006002
    str_iv_run_dir        (recession, theta_p, theta) <U60 'marconi/spice2/new_sheath_exp/flat_flush/alpha_yz_-3.0' ... 'marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-30.0'
    str_iv_voltage_min    (recession, theta_p, theta) float64 -9.95 ... -9.95
    str_iv_voltage_max    (recession, theta_p, theta) float64 -2.05 ... -2.05
    norm_iv_chi2          (recession, theta_p, theta) float64 138.3 ... 153.5
    norm_iv_reduced_chi2  (recession, theta_p, theta) float64 1.281 ... 1.073
    norm_iv_I_sat         (recession, theta_p, theta) float64 -6.476 ... -72.86
    norm_iv_d_I_sat       (recession, theta_p, theta) float64 0.1268 ... 0.7461
    norm_iv_a             (recession, theta_p, theta) float64 0.1512 ... 0.04027
    norm_iv_d_a           (recession, theta_p, theta) float64 0.01033 ... 0.003633
    norm_iv_voltage_min   (recession, theta_p, theta) float64 -5.454 ... -7.219
    norm_iv_voltage_max   (recession, theta_p, theta) float64 -0.004398 ... -0.01927
    ion_chi2              (recession, theta_p, theta) float64 127.9 ... 196.4
    ion_reduced_chi2      (recession, theta_p, theta) float64 1.184 ... 1.373
    ion_I_sat             (recession, theta_p, theta) float64 6.465 ... 79.16
    ion_d_I_sat           (recession, theta_p, theta) float64 0.04734 ... 0.4406
    ion_a                 (recession, theta_p, theta) float64 0.1515 ... 0.01446
    ion_d_a               (recession, theta_p, theta) float64 0.004415 ... 0.00207
    ion_voltage_min       (recession, theta_p, theta) float64 0.01708 ... 0.05173
    ion_voltage_max       (recession, theta_p, theta) float64 3.569 ... 4.404
    current               (recession, theta_p, theta, voltage) float64 -10.29 ... 915.7
    current_e             (recession, theta_p, theta, voltage) float64 -0.02646 ... -918.8
    current_i             (recession, theta_p, theta, voltage) float64 10.32 ... 3.105
    d_current             (recession, theta_p, theta, voltage) float64 0.2479 ... 6.614
    d_current_e           (recession, theta_p, theta, voltage) float64 0.01171 ... 6.695
    d_current_i           (recession, theta_p, theta, voltage) float64 0.2477 ... 0.495
    time                  (recession, theta_p, theta, voltage) float64 0.0 ... 2.581
    None                  (recession) float64 0.0 0.001

## Add Probe Data

In [62]:
combined_ds['theta_p'] = theta_p
combined_ds['recession'] = d_perps
combined_ds = combined_ds.assign_coords(
    theta_p_rads=np.radians(combined_ds.theta_p),
    theta_rads=np.radians(combined_ds.theta)
).drop(None)

In [63]:
combined_ds

<xarray.Dataset>
Dimensions:               (recession: 2, theta: 9, theta_p: 2, voltage: 401)
Coordinates:
  * recession             (recession) float64 0.0 0.001
    theta_p_desc          (theta_p) <U6 'flat' 'angled'
  * theta_p               (theta_p) float64 0.0 10.0
  * theta                 (theta) float64 3.0 4.0 5.0 6.0 ... 12.0 20.0 30.0
  * voltage               (voltage) float64 -9.95 -9.9 -9.85 ... 9.95 10.0 10.05
    recession_desc        (recession) <U8 'flush' 'recessed'
    theta_p_rads          (theta_p) float64 0.0 0.1745
    theta_rads            (theta) float64 0.05236 0.06981 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (recession, theta_p, theta) float64 6.743e+03 ... 159.0
    str_iv_reduced_chi2   (recession, theta_p, theta) float64 43.5 ... 1.026
    str_iv_I_sat          (recession, theta_p, theta) float64 -11.35 ... -71.91
    str_iv_d_I_sat        (recession, theta_p, theta) float64 3.13 ... 1.296
    str_iv_a              (recession, theta_p, theta) float64 1.868e-15 ... 0.04223
    str_iv_d_a            (recession, theta_p, theta) float64 0.12 ... 0.005092
    str_iv_T_e            (recession, theta_p, theta) float64 1.995 ... 0.9402
    str_iv_d_T_e          (recession, theta_p, theta) float64 0.3035 ... 0.02061
    str_iv_V_f            (recession, theta_p, theta) float64 -4.721 ... -2.776
    str_iv_d_V_f          (recession, theta_p, theta) float64 0.05557 ... 0.006002
    str_iv_run_dir        (recession, theta_p, theta) <U60 'marconi/spice2/new_sheath_exp/flat_flush/alpha_yz_-3.0' ... 'marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-30.0'
    str_iv_voltage_min    (recession, theta_p, theta) float64 -9.95 ... -9.95
    str_iv_voltage_max    (recession, theta_p, theta) float64 -2.05 ... -2.05
    norm_iv_chi2          (recession, theta_p, theta) float64 138.3 ... 153.5
    norm_iv_reduced_chi2  (recession, theta_p, theta) float64 1.281 ... 1.073
    norm_iv_I_sat         (recession, theta_p, theta) float64 -6.476 ... -72.86
    norm_iv_d_I_sat       (recession, theta_p, theta) float64 0.1268 ... 0.7461
    norm_iv_a             (recession, theta_p, theta) float64 0.1512 ... 0.04027
    norm_iv_d_a           (recession, theta_p, theta) float64 0.01033 ... 0.003633
    norm_iv_voltage_min   (recession, theta_p, theta) float64 -5.454 ... -7.219
    norm_iv_voltage_max   (recession, theta_p, theta) float64 -0.004398 ... -0.01927
    ion_chi2              (recession, theta_p, theta) float64 127.9 ... 196.4
    ion_reduced_chi2      (recession, theta_p, theta) float64 1.184 ... 1.373
    ion_I_sat             (recession, theta_p, theta) float64 6.465 ... 79.16
    ion_d_I_sat           (recession, theta_p, theta) float64 0.04734 ... 0.4406
    ion_a                 (recession, theta_p, theta) float64 0.1515 ... 0.01446
    ion_d_a               (recession, theta_p, theta) float64 0.004415 ... 0.00207
    ion_voltage_min       (recession, theta_p, theta) float64 0.01708 ... 0.05173
    ion_voltage_max       (recession, theta_p, theta) float64 3.569 ... 4.404
    current               (recession, theta_p, theta, voltage) float64 -10.29 ... 915.7
    current_e             (recession, theta_p, theta, voltage) float64 -0.02646 ... -918.8
    current_i             (recession, theta_p, theta, voltage) float64 10.32 ... 3.105
    d_current             (recession, theta_p, theta, voltage) float64 0.2479 ... 6.614
    d_current_e           (recession, theta_p, theta, voltage) float64 0.01171 ... 6.695
    d_current_i           (recession, theta_p, theta, voltage) float64 0.2477 ... 0.495
    time                  (recession, theta_p, theta, voltage) float64 0.0 ... 2.581

In [64]:
combined_ds.to_netcdf('new_sheath_exp_ivs.nc')

In [68]:
# sanity check

combined_ds.sel(theta=3.0, theta_p=0.0, recession=0.0).fit_run_dir

AttributeError: 'Dataset' object has no attribute 'fit_run_dir'

In [86]:
probe_desig = {
    'angled': {'theta_p':10.0, 'recession':0.0},
    'angled_semi-recessed': {'theta_p':10.0, 'recession':5.0e-4},
    'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3},
    'semi-angled': {'theta_p':5.0, 'recession':0.0},
    'semi-angled_semi-recessed': {'theta_p':5.0, 'recession':5.0e-4},
    'semi-angled_recessed': {'theta_p':5.0, 'recession':1.0e-3},
    'flat': {'theta_p':0.0, 'recession':0.0},
    'flat_semi-recessed': {'theta_p':0.0, 'recession':5.0e-4},
    'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3},
}

(2020-10-07) Everything above here has been made to make the netcdf with homogenised IV characteristics and several fits. The below was used to prototype these fitting routines and find the ones which worked best to get sheath expansion params out. 

This analysis is therefore carried on in a second notebook, with teh later parts of this spreadsheet left 'as is' for posterity and continued iteration. 

---

# Make some plots!



In [118]:
fig, ax = plt.subplots()
# combined_ds['fit_T_e'].plot(x='angle', linestyle='none', marker='.', mfc='none')
ax.errorbar('theta', 'fit_T_e', yerr='fit_d_T_e', data=combined_ds.sel(theta_p=10.0, recession=0.0), 
            linestyle='none', marker='.', mfc='none')

<ErrorbarContainer object of 3 artists>

In [20]:
fig, ax = plt.subplots(4, sharex=True, figsize=[4, 8])

plot_vars = ['fit_{}T_e', 'fit_{}I_sat', 'fit_{}a', 'fit_{}V_f']

for probe in combined_ds.probe.values:
    if probe == 'flat_recessed':
        continue
    plot_ds = combined_ds.sel(probe=probe)
    for i, plot_var in enumerate(plot_vars):
        ax[i].errorbar('theta', plot_var.format(''), yerr=plot_var.format('d_'), data=plot_ds, 
                       linestyle='none', marker='.', mfc='none', label=probe)
        ax[i].set_ylabel(r'${}$'.format(plot_var.split('{}')[-1]))

    # ax[3].plot('angle', 'fit_reduced_chi2', data=combined_ds, linestyle='none', marker='.', mfc='none')
    # ax[3].set_ylabel(r'$\chi_{\nu}^{2}$')
    ax[3].plot('theta', 'fit_voltage_max', data=plot_ds, linestyle='none', marker='.', mfc='none')

    ax[3].set_xlabel(r'$\theta$ [$\degree$]')
    
ax[0].legend()
ax[1].legend()
ax[2].legend()

fig.tight_layout()

In [119]:
fig, ax = plt.subplots(2, sharex=True, figsize=[4, 8])

ax[0].plot('angle', 'fit_V_f', data=combined_ds, linestyle='none', marker='.', mfc='none')
ax[0].plot('angle', 'fit_voltage_max', data=combined_ds, linestyle='none', marker='.', mfc='none')

ax[1].plot(combined_ds.angle.values, (combined_ds['fit_voltage_max'] - combined_ds.fit_V_f).values, 
           linestyle='none', marker='.', mfc='none')
ax[1].axhline(y=3, **c.AX_LINE_DEFAULTS)

ValueError: x and y must have same first dimension, but have shapes (1,) and (3, 3, 6)

In [287]:
fig, ax = plt.subplots(3, figsize=[6,10], sharex=True)

probe = 'flat'


voltage_range = np.array([-9.95, 0.1])

plot_ds = combined_ds.sortby('theta').sel(voltage=slice(*voltage_range), **probe_desig[probe])

plot_ds['current'].plot.line(x='voltage', hue='theta', ax=ax[0], linestyle='none', marker='+')
plot_ds['current_i'].plot.line(x='voltage', hue='theta', ax=ax[1])
plot_ds['current_e'].plot.line(x='voltage', hue='theta', ax=ax[2])

fitter = fts.FullIVFitter()
for theta in plot_ds.theta.values:
    ds = plot_ds.sel(theta=theta)
    dummy_voltage = np.linspace(ds.fit_voltage_min.values, ds.fit_voltage_max.values, 1000)
    fit_current = fitter.fit_function(dummy_voltage, 
                                      ds.fit_I_sat.values, 
                                      ds.fit_a.values, 
                                      ds.fit_T_e.values, 
                                      ds.fit_V_f.values)
    ax[0].plot(dummy_voltage, fit_current)
    ax[0].axvline(x=ds.fit_voltage_max.values, **c.AX_LINE_DEFAULTS)
    
ax[0].set_xlim(voltage_range[0] - 0.5, voltage_range[1] + 0.5)

fig.tight_layout()

/home/jleland/coding/projects/flopter/flopter/core/fitters.py:246: RuntimeWarning: invalid value encountered in less_equal
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))


In [274]:
# The great Ion current plot with fits and a comparison to the fit to the total current IV

probe='flat'

fig = plt.figure(figsize=[10,7])
gs = fig.add_gridspec(2, 2)
ax0 = fig.add_subplot(gs[0:, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 1], sharex=ax1)
plt.setp(ax1.get_xticklabels(), visible=False)

ax = [ax0, ax1, ax2]

plot_ds = combined_ds.sel(**probe_desig[probe]).sortby('theta').sel(voltage=slice(-9.95, -6.0))

cols=[
    'probe',
    'angle',
    'I_sat',
    'd_I_sat',
    'a',
    'd_a',
    'reduced_chi2'
]
ion_fits = pd.DataFrame(columns=cols)

for theta in plot_ds.theta.values:
    sim_ds = plot_ds.sel(theta=theta)
    ax[0].errorbar(np.float_power(np.abs(sim_ds['voltage']), .75), 
                   sim_ds['current_i'], yerr=sim_ds['d_current_i'], 
                   label=r'${{\theta}} = $ {}'.format(theta))
    
    i_fitter = fts.IonCurrentSEFitter()
    fd = i_fitter.fit(np.float_power(np.abs(sim_ds['voltage'].values), .75), 
                      sim_ds['current_i'].values,
                      sigma=sim_ds['d_current_i'].values)
    
    ax[0].plot(*fd.get_fit_plottables(), color='grey')
    
    fd_dict = {k: v for k, v in fd.to_dict().items() 
               if k in cols}
    ion_fits = ion_fits.append({
        'probe': probe,
        'theta': theta,
        **fd_dict,
    }, ignore_index=True)

ax[0].set_ylabel(r'$\hat{I_{i}}$')
ax[0].set_xlabel(r'$|V|^{\frac{3}{4}}$')
ax[0].legend()

inner_ax = plt.axes([0.15, 0.45, .14, .2])
(-plot_ds['current_e']).plot.line(x='voltage', 
                                  hue='theta', 
                                  ax=inner_ax)
inner_ax.set_ylabel('')
inner_ax.set_xlabel('')
inner_ax.set_title(r'$I_e$')
inner_ax.legend().remove()

fig.suptitle(f'Ion Current Analysis for {probe} Probe')

ax[1].errorbar('theta', 'a', yerr='d_a', data=ion_fits, fmt='s', mfc='none', label=r'$I_{i}$')
ax[1].errorbar('theta', 'fit_a', yerr='fit_d_a', data=plot_ds, fmt='^', mfc='none', label=r'$I_{tot}$')
ax[1].errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
ax[1].legend()
ax[1].set_ylabel(r'$a$')
ax[1].set_ylim(1e-3, 5e-1)
ax[1].set_xlim(-0.5, 30.5)

ax[2].errorbar('theta', 'I_sat', yerr='d_I_sat', data=ion_fits, fmt='^', mfc='none', label=r'$I_{i}$')
ax[2].errorbar('theta', 'fit_I_sat', yerr='fit_d_I_sat', data=plot_ds, fmt='^', mfc='none', 
               label=r'$I_{tot}$')
ax[2].legend()
ax[2].set_ylabel(r'$I_{sat}$')
ax[2].set_xlabel(r'$\theta$')

gs.tight_layout(fig, rect=[0, 0, 1.0, 0.95])

No handles with labels found to put in legend.
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [260]:
# fig, ax = plt.subplots()

# ax.errorbar(sim_ds['voltage'], sim_ds['current_e'], yerr=sim_ds['d_current_e'], label='data')

# # for temperature in np.logspace(-0, 2, 3):
# #     ax.plot(sim_ds['voltage'], (0.0 - np.exp(-(-10.0 - sim_ds['voltage']) / temperature)), 
# #             label=f'{temperature}')
# ax.plot(sim_ds['voltage'], (0.0 - np.exp(-(-10.0 - sim_ds['voltage']) / 1.5)), 
#         label=f'{1.5}')
# ax.legend()

In [213]:
splopter =  spl.Splopter(spl_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True)

In [299]:
parser = inp.InputParser(input_filename=spl_path / 'input.inp')
denormaliser = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser) #, dimensions=3)
print(splopter.tdata.dt)
print(denormaliser.K)

[[0.00033962]]
55263494.10840861


In [252]:
dV = parser.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser.getfloat('mks', 'mks_te')
n_0 = parser.getfloat('mks', 'mks_n0')

L = parser.getfloat('rectangle2', 'yhigh') - parser.getfloat('rectangle2', 'ylow')
g = parser.getfloat('rectangle2', 'ylow') - parser.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
theta_p = np.radians(10.0)

I_0 = c.ELEM_CHARGE * n_0 * lpu.sound_speed(T_e) * L * lambda_D * np.sin(theta)

print(f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {theta_p} \n"
      f"I_0 = {I_0}")

L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 
I_0 = -24.432222112932


In [224]:
def decompose_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (((L + g) * np.tan(theta)) 
                                       + (L * np.tan(theta_p)) - d_perp))
    x = np.tan(theta) + np.tan(theta_p)
    return x, y

def decompose_altnew_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (((L + g) * np.tan(theta)) 
                                       + (L * np.tan(theta_p)) - d_perp))
    x = np.tan(theta) + np.tan(theta_p)
    return x, y

def decompose_sheath_exp_param(a, theta, L, g):
    y = a * (L + g) * np.sqrt(np.sin(theta))
    x = np.cos(theta) / np.sin(theta)
    return x, y


In [286]:
probe = 'flat'

plot_ds = combined_ds.sel(**probe_desig[probe]) #, theta=slice(1,10))

# array_1 = (plot_ds['fit_a'] * np.sqrt(np.sin(np.radians(plot_ds['theta']))) 
#            * (((L + g) * np.tan(np.radians(plot_ds['theta']))) + (L * np.tan(theta_p))))
# array_2 = np.tan(np.radians(plot_ds['theta'])) + np.tan(theta_p)
x, y = decompose_new_sheath_exp_param(plot_ds['fit_a'], plot_ds['theta_rads'], L, g, 
                                      plot_ds['recession'], plot_ds['theta_p_rads'])
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]

x_1, y_1 = decompose_sheath_exp_param(plot_ds['fit_a'], plot_ds['theta_rads'], L, g)
x_1 = x_1[~np.isnan(y_1)]
y_1 = y_1[~np.isnan(y_1)]


sl_fitter = fts.StraightLineFitter()
sl_fit_data = sl_fitter.fit(x, y)
sl_fit_data_1 = sl_fitter.fit(x_1, y_1)

fig, ax = plt.subplots()
ax.plot(x, y, '.')
ax.plot(x_1, y_1, 'o')
ax.plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
ax.plot(*sl_fit_data_1.get_fit_plottables(), label=sl_fit_data_1.get_param_str())
ax.legend()



In [226]:
probe = 'angled'

plot_ds = combined_ds.sel(probe=probe) #, theta=slice(1,10))

array_x

sl_fitter = fts.StraightLineFitter()
sl_fit_data = sl_fitter.fit(array_x, array_y)

fig, ax = plt.subplots()
ax.plot(array_x, array_y, '.')
ax.plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
ax.legend()

ValueError: dimensions or multi-index levels ['probe'] do not exist

In [300]:
denormaliser(1, 'current')

1.9980219358841151

So getting weird negative slopes in these angle graphs, so I'm going to try denormalising properly and then trying the slightly altered fit from Bergmann's paper.

In [301]:
combined_ds['norm_current'] = denormaliser(combined_ds['current'], 'current')
combined_ds['norm_d_current'] = denormaliser(combined_ds['d_current'], 'current')

combined_ds['norm_current_i'] = denormaliser(combined_ds['current_i'], 'current')
combined_ds['norm_d_current_i'] = denormaliser(combined_ds['d_current_i'], 'current')

combined_ds['norm_current_e'] = denormaliser(combined_ds['current_e'], 'current')
combined_ds['norm_d_current_e'] = denormaliser(combined_ds['d_current_e'], 'current')

combined_ds.assign_coords(norm_voltage=denormaliser(combined_ds['voltage'], 'potential'))
# combined_ds['V'] = denormaliser(combined_ds['voltage'] - combined_ds['fit_V_f'], 'potential')


<xarray.Dataset>
Dimensions:           (recession: 3, theta: 6, theta_p: 3, voltage: 401)
Coordinates:
    theta_p_desc      (theta_p) <U11 'flat' 'semi-angled' 'angled'
  * voltage           (voltage) float64 -9.95 -9.9 -9.85 ... 9.95 10.0 10.05
  * theta             (theta) float64 1.0 2.0 3.0 5.0 10.0 30.0
    recession_desc    (recession) <U13 'flush' 'semi-recessed' 'recessed'
  * theta_p           (theta_p) float64 0.0 5.0 10.0
  * recession         (recession) float64 0.0 0.0005 0.001
    theta_p_rads      (theta_p) float64 0.0 0.08727 0.1745
    theta_rads        (theta) float64 0.01745 0.03491 0.05236 ... 0.1745 0.5236
    norm_voltage      (voltage) float64 -49.75 -49.5 -49.25 ... 49.75 50.0 50.25
Data variables:
    fit_chi2          (recession, theta_p, theta) float64 438.9 nan ... 150.7
    fit_reduced_chi2  (recession, theta_p, theta) float64 4.026 nan ... 1.077
    fit_I_sat         (recession, theta_p, theta) float64 -3.649 nan ... -72.9
    fit_d_I_sat       (recession, theta_p, theta) float64 0.5565 nan ... 1.781
    fit_a             (recession, theta_p, theta) float64 1.089e-20 ... 0.04027
    fit_d_a           (recession, theta_p, theta) float64 0.05543 ... 0.00634
    fit_T_e           (recession, theta_p, theta) float64 1.559 nan ... 0.9725
    fit_d_T_e         (recession, theta_p, theta) float64 0.2165 nan ... 0.04571
    fit_V_f           (recession, theta_p, theta) float64 -4.32 nan ... -2.764
    fit_d_V_f         (recession, theta_p, theta) float64 0.03581 ... 0.01809
    fit_run_dir       (recession, theta_p, theta) <U67 'marconi/spice2/sheath_exp/flat/alpha_yz_-1.0' ... 'marconi/spice2/sheath_exp/angled_recessed/alpha_yz_-30.0'
    fit_voltage_min   (recession, theta_p, theta) float64 -9.95 -9.95 ... -9.95
    fit_voltage_max   (recession, theta_p, theta) float64 -4.35 10.0 ... -2.8
    current           (recession, theta_p, theta, voltage) float64 -3.703 ... 925.9
    current_e         (recession, theta_p, theta, voltage) float64 -0.001758 ... -928.5
    current_i         (recession, theta_p, theta, voltage) float64 3.705 ... 2.579
    d_current         (recession, theta_p, theta, voltage) float64 0.08342 ... 6.024
    d_current_e       (recession, theta_p, theta, voltage) float64 0.001758 ... 6.075
    d_current_i       (recession, theta_p, theta, voltage) float64 0.08333 ... 0.4734
    time              (recession, theta_p, theta, voltage) float64 0.0 ... 2.581
    norm_current      (recession, theta_p, theta, voltage) float64 -7.399 ... 1.85e+03
    norm_d_current    (recession, theta_p, theta, voltage) float64 0.1667 ... 12.04
    norm_current_i    (recession, theta_p, theta, voltage) float64 7.402 ... 5.153
    norm_d_current_i  (recession, theta_p, theta, voltage) float64 0.1665 ... 0.9458
    norm_current_e    (recession, theta_p, theta, voltage) float64 -0.003512 ... -1.855e+03
    norm_d_current_e  (recession, theta_p, theta, voltage) float64 0.003512 ... 12.14
    V                 (voltage, recession, theta_p, theta) float64 -28.15 ... 64.07

In [228]:
dummy_theta = np.linspace(0.00001, 45.0, 5000)
calced_a = lpu.calc_sheath_expansion_coeff(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    c_1=0.5,
    c_2=0.6,
)

alt_calced_a = lpu.calc_new_sheath_expansion_param(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    0.0,
    np.radians(10.0),
    c_1=0.5,
    c_2=0.6,
)

In [338]:
# The altered Ion current plot

probe='flat'

fig = plt.figure(figsize=[10,6])
gs = fig.add_gridspec(2, 2)
ax0 = fig.add_subplot(gs[0:, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 1], sharex=ax1)
plt.setp(ax1.get_xticklabels(), visible=False)

ax = [ax0, ax1, ax2]

fig_iv, ax_iv = plt.subplots()

plot_ds = combined_ds.sel(**probe_desig[probe]).sortby('theta').sel(voltage=slice(-9.95, 0.0))

cols=[
    'probe',
    'angle',
    'I_sat',
    'd_I_sat',
    'a',
    'd_a',
    'reduced_chi2'
]
norm_ion_fits = pd.DataFrame(columns=cols)

for theta in plot_ds.theta.values:
    sim_ds = plot_ds.sel(theta=theta)
    
    V_f = fts.IVFitter.find_floating_pot(sim_ds['voltage'], sim_ds['current'])
    norm_V_f = denormaliser(V_f, 'potential')
    print(V_f)
    
    sim_ds = sim_ds.sel(voltage=slice(-9.95, V_f))
        
    ax[0].errorbar(np.float_power(np.abs(sim_ds['voltage'] - V_f), .75), 
                   sim_ds['current_i'], yerr=sim_ds['d_current_i'])
    
    i_fitter = fts.IonCurrentSEFitter()
    fd = i_fitter.fit(np.float_power(np.abs(sim_ds['voltage'] - V_f).values, .75), 
                      sim_ds['current_i'].values,
                      sigma=sim_ds['d_current_i'].values)
    
    ax[0].plot(*fd.get_fit_plottables())
    I_0 = fd.get_isat()
    
    ax_iv.errorbar(sim_ds['voltage'] - V_f, sim_ds['current']/I_0, yerr=sim_ds['d_current']/I_0, 
                   label=r'${{\theta}} = $ {}'.format(theta))
    ax_iv.legend()
    
    fd_dict = {k: v for k, v in fd.to_dict().items() 
               if k in cols}
    norm_ion_fits = norm_ion_fits.append({
        'probe': probe,
        'theta': theta,
        **fd_dict,
    }, ignore_index=True)

ax[0].set_ylabel(r'$I_{i}$')
ax[0].set_xlabel(r'$|V|^{\frac{3}{4}}$')

fig.suptitle(f'Ion Current Analysis for {probe.capitalize()} Probe')


ax[1].errorbar('theta', 'a', yerr='d_a', data=norm_ion_fits, fmt='s', mfc='none', label=r'$I_{i}$')
# ax[1].errorbar('theta', 'fit_a', yerr='fit_d_a', data=plot_ds, fmt='^', mfc='none', label=r'$I_{tot}$')
ax[1].errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
ax[1].errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
               color='orange', linewidth=0.8, alpha=0.6)
ax[1].legend()
ax[1].set_ylabel(r'$a$')
ax[1].set_ylim(1e-3, 5e-1)
ax[1].set_xlim(-0.5, 30.5)

ax[2].errorbar('theta', 'I_sat', yerr='d_I_sat', data=norm_ion_fits, fmt='^', mfc='none', label=r'$I_{i}$')
# ax[2].errorbar('theta', 'fit_I_sat', yerr='fit_d_I_sat', data=-plot_ds, fmt='^', mfc='none', 
#                label=r'$I_{tot}$')
ax[2].legend()
ax[2].set_ylabel(r'$I_{sat}$')
ax[2].set_xlabel(r'$\theta$')

gs.tight_layout(fig, rect=[0, 0, 1.0, 0.95])

-4.309590805138028
-4.828686741506997
-4.4956018612042445
-4.403934375163689
-4.0109978067524965
-2.9011683470668834


In [327]:
class PartialIVFitter(fts.IVFitter):
    """
    IV Fitter implementation utilising a partial, 4 parameter IV Curve fitting method.
    """
    def __init__(self):
        super().__init__()
        self._param_labels = {
            c.ION_SAT: 0,
            c.SHEATH_EXP: 1,
            c.ELEC_TEMP: 2,
        }
        self.default_values = (30.0, 0.0204, 1)
        self.default_bounds = (
            (-np.inf,      0,       0),
            ( np.inf,    1.5,  np.inf)
        )
        self.name = '4 Parameter Fit'

    def fit_function(self, v, *parameters):
        I_0 = parameters[self._param_labels[c.ION_SAT]]
        a = parameters[self._param_labels[c.SHEATH_EXP]]
        T_e = parameters[self._param_labels[c.ELEC_TEMP]]
        V = -v / T_e

        return I_0 * (1 - np.exp(-V) + np.where(v <= 0, (a * np.float_power(np.absolute(V), [0.75])), 0))

    def get_a_index(self):
        return self._param_labels[c.SHEATH_EXP]
    
class NormalisedIVFitter(fts.IVFitter):
    """
    IV Fitter implementation utilising a partial, 4 parameter IV Curve fitting method.
    """
    def __init__(self):
        super().__init__()
        self._param_labels = {
            c.ION_SAT: 0,
            c.SHEATH_EXP: 1,
        }
        self.default_values = (30.0, 0.0204)
        self.default_bounds = (
            (-np.inf,      0),
            ( np.inf,    1.5)
        )
        self.name = 'Normalised Parameter Fit'

    def fit_function(self, v, *parameters):
        I_0 = parameters[self._param_labels[c.ION_SAT]]
        a = parameters[self._param_labels[c.SHEATH_EXP]]
        
        return I_0 * (1 - np.exp(v) + np.where(v <= 0, (a * np.float_power(np.absolute(v), [0.75])), 0))

    def get_a_index(self):
        return self._param_labels[c.SHEATH_EXP]


In [330]:
# The denormalised IV Fitting

fig, ax = plt.subplots()

probe='flat'
plot_ds = combined_ds.sel(**probe_desig[probe]).sortby('theta').sel(voltage=slice(-9.95, 2.0))

cols=[
    'probe',
    'angle',
    'I_sat',
    'd_I_sat',
    'a',
    'd_a',
    'T_e',
    'd_T_e',
    'V_f',
    'd_V_f',
    'reduced_chi2'
]
iv_fits = pd.DataFrame(columns=cols)

calced_I_0s = []
fitted_I_0s = []

for theta in plot_ds.theta.values:
    sim_ds = plot_ds.sel(theta=theta)
    I_0 = c.ELEM_CHARGE * n_0 * lpu.sound_speed(T_e) * L * lambda_D * np.sin(np.radians(theta))
    calced_I_0s.append(I_0)
    
    V_f = fts.IVFitter.find_floating_pot(sim_ds['voltage'], sim_ds['current'])
    
    sim_ds = sim_ds.sel(voltage=slice(-9.95, V_f))
    
    voltage = sim_ds['voltage'].values - V_f
    current = sim_ds['current'].values
    d_current = sim_ds['d_current'].values
    
    iv_fitter = NormalisedIVFitter()
    fd = iv_fitter.fit(voltage, current, sigma=d_current)
    
    I_0 = fd.get_isat()
    fitted_I_0s.append(-I_0)
    
    ax.errorbar(voltage, current/I_0, yerr=d_current/I_0, fmt='x', label=f'{theta}')
    ax.plot(voltage, fd.fit_y / I_0, color='k')
    
    fd_dict = {k: v for k, v in fd.to_dict().items() 
               if k in cols}
    iv_fits = iv_fits.append({
        'probe': probe,
        'theta': theta,
        **fd_dict,
    }, ignore_index=True)

ax.legend(title=r'$\theta$')
ax.set_ylabel(r'$I / I_0$')
ax.set_xlabel(r'$V$')

Text(0.5, 0, '$V$')

In [306]:
fig, (ax, ax1) = plt.subplots(2, sharex=True)

ax.plot(plot_ds.theta.values, calced_I_0s, label='calced')
ax.plot(plot_ds.theta.values, fitted_I_0s, label='fitted')
ax.legend()

ax1.plot(plot_ds.theta.values, np.array(calced_I_0s) / np.array(fitted_I_0s))

NameError: name 'calced_I_0s' is not defined

In [344]:
# probe = 'angled'

plot_ds = combined_ds.sel(**probe_desig[probe]) #, theta=slice(1,10))

array_y = ion_fits['a'] * (L + g) * np.sqrt(np.sin(np.radians(ion_fits['theta'])))
array_x = np.cos(np.radians(ion_fits['theta'])) / np.sin(np.radians(ion_fits['theta']))

array_y_0 = norm_ion_fits['a'] * (L + g) * np.sqrt(np.sin(np.radians(norm_ion_fits['theta'])))
array_x_0 = np.cos(np.radians(norm_ion_fits['theta'])) / np.sin(np.radians(norm_ion_fits['theta']))

array_y_1 = plot_ds['fit_a'] * (L + g) * np.sqrt(np.sin(np.radians(plot_ds['theta'])))
array_x_1 = np.cos(np.radians(plot_ds['theta'])) / np.sin(np.radians(plot_ds['theta']))

array_y_2 = iv_fits['a'] * (L + g) * np.sqrt(np.sin(np.radians(iv_fits['theta'])))
array_x_2 = np.cos(np.radians(iv_fits['theta'])) / np.sin(np.radians(iv_fits['theta']))

array_y_old = flush_df['a'] * (L + g) * np.sqrt(np.sin(np.radians(flush_df['theta'])))
array_x_old = np.cos(np.radians(flush_df['theta'])) / np.sin(np.radians(flush_df['theta']))

array_y_calc = calced_a * (L + g) * np.sqrt(np.sin(np.radians(dummy_theta)))
array_x_calc = np.cos(np.radians(dummy_theta)) / np.sin(np.radians(dummy_theta))

# sl_fitter = fts.StraightLineFitter()
# sl_fit_data = sl_fitter.fit(array_x, array_y)
# sl_fit_data_1 = sl_fitter.fit(array_x_1, array_y_1)
# sl_fit_data_2 = sl_fitter.fit(array_x_2, array_y_2)


fig, ax = plt.subplots(1, 2, figsize=[12,6])

fig.suptitle('Sheath Expansion Fits to Flat-Flush Probe')

# ax[0].errorbar('theta', 'a', yerr='d_a', data=ion_fits, fmt='s', mfc='none', label=r'Ion Fit')
ax[0].errorbar('theta', 'a', yerr='d_a', data=norm_ion_fits, fmt='.', mfc='none', label=r'Norm Ion Fit')
# ax[0].errorbar('theta', 'fit_a', yerr='fit_d_a', data=plot_ds, fmt='o', mfc='none', 
#                label=r'IV Fit')
ax[0].errorbar('theta', 'a', yerr='d_a', data=iv_fits, fmt='^', mfc='none', label=r'Norm IV Fit')
ax[0].errorbar('theta', 'a', yerr='d_a', data=flush_df, fmt='*', mfc='none', label=r'Old Sims')

ax[0].errorbar(dummy_theta, calced_a, label=r'Theoretical', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
# ax[0].errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
#                color='orange', linewidth=0.8, alpha=0.6)
ax[0].legend()
ax[0].set_ylabel(r'$a$')
ax[0].set_xlabel(r'$\theta$')
ax[0].set_ylim(-0.01, 5e-1)
ax[0].set_xlim(-0.5, 30.5)


# ax[1].plot(array_x, array_y, 's', mfc='none', label='Ion fit')
ax[1].plot(array_x_0, array_y_0, '.', mfc='none', label='Norm Ion fit')
# ax[1].plot(array_x_1, array_y_1, 'o', mfc='none', label='IV Fit')
ax[1].plot(array_x_2, array_y_2, '^', mfc='none', label='Norm IV Fit')
ax[1].plot(array_x_old, array_y_old, '*', mfc='none', label='Old sim')

ax[1].plot(array_x_calc, array_y_calc, '-', color='r', linewidth=0.8, alpha=0.6, label='Theoretical')

# ax[1].plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
# ax[1].plot(*sl_fit_data_1.get_fit_plottables(), label=sl_fit_data_1.get_param_str())
# ax[1].plot(*sl_fit_data_2.get_fit_plottables(), label=sl_fit_data_2.get_param_str())
ax[1].legend()

ax[1].set_xlabel(r'$\cos{\theta} \; / \; \sin{\theta}$')
ax[1].set_ylabel(r'$a L \sin^{1/2}{\theta}$')
ax[1].set_xlim(-0.5, 60)
ax[1].set_ylim(-0.5, 40)

fig.tight_layout(rect=[0, 0, 1.0, 0.95])

## Compare also to some older simulations

In [153]:
lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
fit_data_filename = 'lowdens_fitdata.csv'

fit_df = pd.read_csv(lowdens_dir / fit_data_filename, index_col=0)
fit_df

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,0.034400,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,0.013990,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,0.004756,0.003758,-3.484159,0.003447
3,angled,4.0,1.023927,0.018306,14.362077,0.343829,0.012245,0.007634,-3.423099,0.007020
4,angled,5.0,1.013085,0.010332,18.032248,0.256156,0.010595,0.004569,-3.457255,0.004239
...,...,...,...,...,...,...,...,...,...,...
83,sprobe,30.0,1.033611,0.010399,62.740506,0.552424,0.012586,0.002549,-2.732093,0.002039
84,sprobe,45.0,1.047113,0.014941,83.666200,1.018863,0.007753,0.003510,-2.690400,0.002807
85,sprobe,60.0,1.006582,0.010704,86.495626,0.719475,0.016005,0.002336,-2.594291,0.002069
86,sprobe,75.0,0.996101,0.012401,87.772275,0.795372,0.017997,0.002498,-2.483807,0.002520


In [154]:
fit_df.groupby('probe').groups

{'angled': Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64'),
 'flush': Int64Index([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
             35, 36, 37, 38],
            dtype='int64'),
 'rearwall': Int64Index([39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53], dtype='int64'),
 'recessed': Int64Index([54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
             71, 72],
            dtype='int64'),
 'sprobe': Int64Index([73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87], dtype='int64')}

In [155]:
flush_df = fit_df.loc[fit_df['probe'] == 'flush']
flush_df = flush_df.loc[flush_df['angle'] <= 30]
# flush_df = flush_df.loc[flush_df['angle'] > 3]
flush_df = flush_df.drop(20)
flush_df = flush_df.rename(columns={'angle':'theta'})
flush_df

,probe,theta,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
18,flush,1.0,1.196922,0.159300,2.005588,0.248067,0.234585,0.045250,-2.824577,0.023557
19,flush,2.0,0.746410,0.041318,3.856090,0.219110,0.189780,0.018324,-2.920495,0.018052
21,flush,4.0,0.862937,0.021906,8.867943,0.236408,0.130579,0.008518,-3.000390,0.007916
22,flush,5.0,0.984287,0.023889,12.246093,0.301755,0.088044,0.007808,-2.990635,0.006228
23,flush,6.0,0.980118,0.021951,14.471464,0.340238,0.076094,0.007440,-3.051633,0.006168
24,flush,7.0,0.940242,0.009109,16.238058,0.168016,0.075870,0.003233,-3.054091,0.002874
25,flush,8.0,1.017323,0.009001,19.388100,0.176069,0.055888,0.002833,-3.014263,0.002248
26,flush,9.0,0.984877,0.007602,21.051741,0.187058,0.060599,0.002850,-3.167852,0.002429
27,flush,10.0,1.011952,0.009473,23.709109,0.223009,0.052364,0.002896,-2.981223,0.002314
28,flush,15.0,1.055231,0.012629,36.369258,0.426445,0.030411,0.003559,-2.932875,0.002739


In [156]:
angled_df = fit_df.loc[fit_df['probe'] == 'angled']
angled_df = angled_df.loc[angled_df['angle'] <= 30]
# flush_df = flush_df.loc[flush_df['angle'] > 3]
# angled_df = angled_df.drop(20)
angled_df = angled_df.rename(columns={'angle':'theta'})
angled_df

,probe,theta,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,3.440025e-02,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,1.398953e-02,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,4.755583e-03,0.003758,-3.484159,0.003447
3,angled,4.0,1.023927,0.018306,14.362077,0.343829,1.224509e-02,0.007634,-3.423099,0.007020
4,angled,5.0,1.013085,0.010332,18.032248,0.256156,1.059543e-02,0.004569,-3.457255,0.004239
5,angled,9.0,1.051815,0.008428,32.520786,0.336018,9.930011e-03,0.003315,-3.362492,0.002789
6,angled,10.0,1.033696,0.010810,35.128063,0.458634,1.811731e-02,0.004147,-3.335452,0.003589
7,angled,15.0,1.088697,0.025033,54.814766,1.440419,4.923012e-18,0.008144,-3.217063,0.006442
8,angled,20.0,1.067608,0.011998,64.130095,0.747523,1.280170e-02,0.003548,-3.038085,0.002769
9,angled,25.0,1.058848,0.009242,75.869397,0.611659,1.295753e-02,0.002377,-2.838315,0.001848


In [157]:
fig, ax = plt.subplots()
ax.errorbar('theta', 'a', yerr='d_a', data=flush_df, fmt='x')
ax.errorbar('theta', 'a', yerr='d_a', data=angled_df, fmt='s', mfc='none')

ax.errorbar(dummy_theta, calced_a, label=r'Theoretical', fmt='-', 
            color='r', linewidth=0.8, alpha=0.6)
ax.errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
            color='orange', linewidth=0.8, alpha=0.6)

ax.set_ylim(0, 5e-1)
ax.set_xlim(-0.5, 30.5)

(-0.5, 30.5)

In [159]:
plot_ds = combined_ds.sel(**probe_desig[probe]) #, theta=slice(1,10))

array_y = flush_df['a'] * (L) * np.sqrt(np.sin(np.radians(flush_df['theta'])))
array_x = np.cos(np.radians(flush_df['theta'])) / np.sin(np.radians(flush_df['theta']))

array_y_calc = calced_a * (L) * np.sqrt(np.sin(np.radians(dummy_theta)))
array_x_calc = np.cos(np.radians(dummy_theta)) / np.sin(np.radians(dummy_theta))

# sl_fitter = fts.StraightLineFitter()
# sl_fit_data = sl_fitter.fit(array_x, array_y)
# sl_fit_data_1 = sl_fitter.fit(array_x_1, array_y_1)
# sl_fit_data_2 = sl_fitter.fit(array_x_2, array_y_2)


fig, ax = plt.subplots(1, 2, figsize=[12,6])

fig.suptitle('Sheath Expansion Fits to Flat-Flush Probe')

ax[0].errorbar('theta', 'a', yerr='d_a', data=flush_df, fmt='s', mfc='none', label=r'Old Sim Fit')

ax[0].errorbar(dummy_theta, calced_a, label=r'Theoretical', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)
# ax[0].errorbar(dummy_theta, alt_calced_a, label=r'calc - new', fmt='-', 
#                color='orange', linewidth=0.8, alpha=0.6)
ax[0].legend()
ax[0].set_ylabel(r'$a$')
ax[0].set_xlabel(r'$\theta$')
ax[0].set_ylim(0, 5e-1)
ax[0].set_xlim(-0.5, 30.5)


ax[1].plot(array_x, array_y, 's', mfc='none', label='Old sim fit')

ax[1].plot(array_x_calc, array_y_calc, '-', color='r', linewidth=0.8, alpha=0.6, label='Theoretical')

# ax[1].plot(*sl_fit_data.get_fit_plottables(), label=sl_fit_data.get_param_str())
# ax[1].plot(*sl_fit_data_1.get_fit_plottables(), label=sl_fit_data_1.get_param_str())
# ax[1].plot(*sl_fit_data_2.get_fit_plottables(), label=sl_fit_data_2.get_param_str())
ax[1].legend()

ax[1].set_xlabel(r'$\cos{\theta} \; / \; \sin{\theta}$')
ax[1].set_ylabel(r'$a L \sin^{1/2}{\theta}$')
ax[1].set_xlim(-0.5, 60)
ax[1].set_ylim(-0.55, 35)

fig.tight_layout(rect=[0, 0, 1.0, 0.95])

# Making the IV characteristic plot in Bergmann's 1994 paper

In [247]:
def a_0(theta):
    return lpu.calc_sheath_expansion_coeff(
        T_e, 
        n_0, 
        L * lambda_D, 
        g * lambda_D, 
        np.radians(theta),
        c_1=0.5, 
        c_2=0.6
    )

In [249]:
fig, ax = plt.subplots()
dummy_voltage_2 = np.linspace(-30, 0, 9950)

I_0 = c.ELEM_CHARGE * n_0 * lpu.sound_speed(T_e) * L * lambda_D * np.sin(np.radians(90.0))

piv_fitter = PartialIVFitter()
dummy_current = piv_fitter.fit_function(dummy_voltage_2, *[I_0, 0.25, 1.0])

ax.plot(dummy_voltage_2, -piv_fitter.fit_function(dummy_voltage_2, *[I_0, a_0(90.0), 1.0]), 
        color='k', linestyle='--', label=f'Ideal')
for theta in [30.0, 10.0, 5.0, 3.0, 2.0, 1.0]:
    ax.plot(dummy_voltage_2, -piv_fitter.fit_function(dummy_voltage_2, *[I_0, a_0(theta), 1.0]), 
            label=r'$\theta = {}$'.format(theta))
ax.legend()
# ax.set_ylim(-4.0,0.0)
ax.set_ylim(-100.0,0.0)
ax.set_xlim(-30.0,0.0)

(-30.0, 0.0)

## Probe Sanity Checks

In [197]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

In [198]:
mag_probes = lpu.MagnumProbes()
mag_probes['L'].d_perp

(0.0003,)

In [199]:
flush_square = lpu.AngledTipProbe(a=5e-3, b=5e-3, L=5e-3, g=1e-3, d_perp=0e-4, 
                                  theta_f=0.0, theta_p=0.0)

In [200]:
alpha = np.radians(10.0)
print(f'get_collection_area() = \t {flush_square.get_collection_area(alpha)}')
print(f'get_2d_collection_length() = \t {flush_square.get_2d_collection_length(alpha)}')
print(f'(L + g) * sin(0) = \t\t {6e-3 * np.sin(alpha)}')
print(f'calc_exposed_lengths() = \t {flush_square.calc_exposed_lengths(alpha)}')
print(f'get_isat() = \t\t\t {flush_square.get_isat(T_e, n_0, alpha)}')

get_collection_area() = 	 [5.20944533e-06]
get_2d_collection_length() = 	 [0.00103925]
(L + g) * sin(0) = 		 0.001041889066001582
calc_exposed_lengths() = 	 (array([0.]), array([0.00017365]))
get_isat() = 			 [0.02583201]


In [204]:
probe_isat = mag_probes['L'].get_isat(T_e, n_0, alpha)
probe_current_pul = lpu.sound_speed(T_e) * n_0 * c.ELEM_CHARGE * ((L + g) * lambda_D) * np.sin(alpha)
print(probe_isat, probe_current_pul, probe_isat/probe_current_pul)

[0.02107866] 5.152803707261383 [0.00409072]


In [205]:
flush_square.get_isat(T_e, n_0, alpha) / probe_current_pul

array([0.00501319])

In [206]:
probe_current_pul * flush_square.a

0.025764018536306915